<a href="https://colab.research.google.com/github/SH22Hwang/KoGES_OP/blob/master/OP_20230403.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 골다공증

다양한 모델로 돌려보았다~

- Traditional ML
  + Linear Model
  + KNN
  + Decison Tree
  + SVM
- GBDT
  + XGBoost
  + LGBM
- DNN
  + MLP
  + SAINT

## 개발환경

- PyTorch
  
## 문제점

- classification 모델인데, 어느 한 쪽으로만 예측하는 경우 발생
  - 알고보니, 예측값을 int64로 취급한 것 같음.
  -



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error, roc_auc_score # roc_auc_score(y_test, y_pred)

from xgboost import XGBClassifier
import lightgbm as lgb

import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 불러오기
# _premeno: 폐경 전 여성
# _postmeno: 폐경 후 여성
X_premeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/X_premeno_20230428.csv', index_col=0)
X_postmeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/X_postmeno_20230428.csv', index_col=0)
y_premeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/y_premeno_20230428.csv', index_col=0)
y_postmeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/y_postmeno_20230428.csv', index_col=0)

In [ ]:
# 데이터 분리하기... 데아터 증강 해야하나??

# Split data into train and test sets
X_premeno_train, X_premeno_test, y_premeno_train, y_premeno_test = train_test_split(X_premeno, y_premeno, test_size=0.2, random_state=415)
X_postmeno_train, X_postmeno_test, y_postmeno_train, y_postmeno_test = train_test_split(X_postmeno, y_postmeno, test_size=0.2, random_state=415)

# Create PyTorch tensors and datasets XGBoost, LGBM, SAINT에서는 사용 안 함.
X_premeno_train_tensor = torch.tensor(X_premeno_train.values).float()
X_premeno_test_tensor = torch.tensor(X_premeno_test.values).float()
y_premeno_train_tensor = torch.tensor(y_premeno_train.values).float()
y_premeno_test_tensor = torch.tensor(y_premeno_test.values).float()

X_postmeno_train_tensor = torch.tensor(X_postmeno_train.values).float()
X_postmeno_test_tensor = torch.tensor(X_postmeno_test.values).float()
y_postmeno_train_tensor = torch.tensor(y_postmeno_train.values).float()
y_postmeno_test_tensor = torch.tensor(y_postmeno_test.values).float()

# Create DataLoader for mini-batch training
# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
print(X_premeno_train.shape, X_premeno_test.shape, y_premeno_train.shape, y_premeno_test.shape)
print(X_postmeno_train.shape, X_postmeno_test.shape, y_postmeno_train.shape, y_postmeno_test.shape)

(1374, 157) (344, 157) (1374, 1) (344, 1)
(2347, 157) (587, 157) (2347, 1) (587, 1)


In [ ]:
y_postmeno

,OP
DIST_ID,
NIH2307137554,0
NIH2307649667,0
NIH2307899915,0
NIH2307202612,0
NIH2307895121,1
...,...
NIH2307591031,1
NIH2307808901,1
NIH2307930134,1


In [ ]:
# Statistical Accuracy Measurement Comprehensive Gift Set
def SAMCGS(y_test, y_pred):
  # 정확도 종합 선물 세트
  accuracy = accuracy_score(y_test, y_pred)
  conf_mat = confusion_matrix(y_test, y_pred)
  # tn, fp, fn, tp = conf_mat.ravel()

  precision = precision_score(y_test, y_pred, zero_division=1) # tp / (tp + fp)
  recall = recall_score(y_test, y_pred, zero_division=1)       # tp / (tp + fn)
  f1 = f1_score(y_test, y_pred, zero_division=1) # 2 * (precision * recall) / (precision + recall)

  print(f'''
  Confusion Matrix:\n{conf_mat}\n
  Accuracy: {accuracy:.3f}
  Precison: {precision:.3f}
  Recall  : {recall:.3f}
  F1 Score: {f1:.3f}'''
  )

  # return precision, recall, f1

## Decision Tree

In [ ]:
# Define the Decision Tree model
class DecisionTree(torch.nn.Module):
    def __init__(self, max_depth):
        super(DecisionTree, self).__init__()
        self.max_depth = max_depth
        self.best_feature = None
        self.best_threshold = None
        self.best_impurity = 1.0

    def fit(self, X, y, depth=0):
        # Stop recursion if maximum depth is reached or all samples have the same label
        if depth == self.max_depth or torch.unique(y).shape[0] == 1:
            mode = torch.mode(y)
            self.label = mode.values[0].item()
            # if mode.count > 1:
            #   print(f"Warning: found {mode.count} samples with tie in class labels")
            return

        # Find the feature and threshold that maximally separate the samples
        self.best_feature = None
        self.best_threshold = None
        self.best_impurity = 1.0
        for feature in range(X.shape[1]):
            for threshold in torch.linspace(X[:, feature].min(), X[:, feature].max(), steps=10):
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                if left_indices.sum() == 0 or right_indices.sum() == 0:
                    continue
                left_labels = y[left_indices]
                right_labels = y[right_indices]
                impurity = left_labels.shape[0] / y.shape[0] * self._compute_gini_impurity(left_labels) + right_labels.shape[0] / y.shape[0] * self._compute_gini_impurity(right_labels)
                if impurity < self.best_impurity:
                    self.best_feature = feature
                    self.best_threshold = threshold
                    self.best_impurity = impurity

        # Stop recursion if no feature and threshold can separate the samples
        if self.best_feature is None:
            self.label = torch.mode(y).values.item()
            return

        # Split the samples based on the best feature and threshold
        left_indices = X[:, self.best_feature] <= self.best_threshold
        right_indices = X[:, self.best_feature] > self.best_threshold

        # Recursively fit the left and right child nodes
        self.left = DecisionTree(max_depth=self.max_depth)
        self.left.fit(X[left_indices], y[left_indices], depth+1)
        self.right = DecisionTree(max_depth=self.max_depth)
        self.right.fit(X[right_indices], y[right_indices], depth+1)

    def _compute_gini_impurity(self, y):
        _, counts = torch.unique(y, return_counts=True)
        proportions = counts / y.shape[0]
        return 1 - (proportions ** 2).sum()

    def predict(self, X):
        y_pred = torch.empty(X.shape[0], dtype=torch.int64)
        for i in range(X.shape[0]):
            node = self
            while hasattr(node, 'left'):
                if X[i, node.best_feature] <= node.best_threshold:
                    node = node.left
                else:
                    node = node.right
            y_pred[i] = node.label
        return y_pred


In [ ]:
# Initialize the model and fit it to the training set
model_premeno = DecisionTree(max_depth=3)
model_premeno.fit(torch.tensor(X_premeno_train_tensor), torch.tensor(y_premeno_train_tensor))

# Test the model on the test set
y_premeno_pred = model_premeno.predict(torch.tensor(X_premeno_test.values))

<ipython-input-47-20d57c73e439>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_premeno.fit(torch.tensor(X_premeno_train_tensor), torch.tensor(y_premeno_train_tensor))


In [ ]:
print("폐경 전 여성, Decision Tree")
SAMCGS(y_premeno_test, y_premeno_pred)

폐경 전 여성, Decision Tree

  Confusion Matrix:
[[330   1]
 [ 13   0]]

  Accuracy: 0.959
  Precison: 0.000
  Recall  : 0.000
  F1 Score: 0.000


In [ ]:
# Initialize the model and fit it to the training set
model_postmeno = DecisionTree(max_depth=3)
model_postmeno.fit(torch.tensor(X_postmeno_train_tensor), torch.tensor(y_postmeno_train_tensor))

# Test the model on the test set
y_postmeno_pred = model_postmeno.predict(torch.tensor(X_postmeno_test.values))

<ipython-input-49-cdcc688a0648>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_postmeno.fit(torch.tensor(X_postmeno_train_tensor), torch.tensor(y_postmeno_train_tensor))


In [ ]:
print("폐경 후 여성, Decision Tree")
SAMCGS(y_postmeno_test, y_postmeno_pred)

폐경 후 여성, Decision Tree

  Confusion Matrix:
[[373  44]
 [160  10]]

  Accuracy: 0.652
  Precison: 0.185
  Recall  : 0.059
  F1 Score: 0.089


## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_premeno = KNeighborsClassifier(n_neighbors=2)
knn_postmeno = KNeighborsClassifier(n_neighbors=2)

knn_premeno.fit(X_premeno_train, y_premeno_train)
knn_postmeno.fit(X_postmeno_train, y_postmeno_train)

y_premeno_pred = knn_premeno.predict(X_premeno_test)
y_postmeno_pred = knn_postmeno.predict(X_postmeno_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
print("폐경 전 여성, KNN")
SAMCGS(y_premeno_test, y_premeno_pred)

폐경 전 여성, KNN

  Confusion Matrix:
[[330   1]
 [ 13   0]]

  Accuracy: 0.959
  Precison: 0.000
  Recall  : 0.000
  F1 Score: 0.000


In [ ]:
print("폐경 후 여성, KNN")
SAMCGS(y_postmeno_test, y_postmeno_pred)

폐경 후 여성, KNN

  Confusion Matrix:
[[385  32]
 [138  32]]

  Accuracy: 0.710
  Precison: 0.500
  Recall  : 0.188
  F1 Score: 0.274


## SVM

In [ ]:
# Define the linear SVM model
class SVM(nn.Module):
    def __init__(self, input_dim):
        super(SVM, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

svm_model_premeno = SVM(X_premeno_train.shape[1])
svm_model_postmeno = SVM(X_postmeno_train.shape[1])

# Define the loss function and optimizer
criterion_premeno = nn.MarginRankingLoss(margin=1)
criterion_postmeno = nn.MarginRankingLoss(margin=1)
optimizer_premeno = optim.SGD(svm_model_postmeno.parameters(), lr=0.01)
optimizer_postmeno = optim.SGD(svm_model_postmeno.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    optimizer_premeno.zero_grad()
    optimizer_postmeno.zero_grad()

    output_premeno = svm_model_premeno(X_premeno_train_tensor).squeeze()
    output_postmeno = svm_model_postmeno(X_postmeno_train_tensor).squeeze()

    loss_premeno = criterion_premeno(output_premeno, y_premeno_train_tensor.squeeze(), torch.ones_like(y_premeno_train_tensor).squeeze())
    loss_premeno.backward()
    loss_postmeno = criterion_postmeno(output_postmeno, y_postmeno_train_tensor.squeeze(), torch.ones_like(y_postmeno_train_tensor).squeeze())
    loss_postmeno.backward()

    optimizer_premeno.step()
    optimizer_postmeno.step()

# Make predictions on the test set
y_premeno_pred = torch.sign(svm_model_premeno(X_premeno_test_tensor).squeeze()).detach().numpy()
y_postmeno_pred = torch.sign(svm_model_postmeno(X_postmeno_test_tensor).squeeze()).detach().numpy()

In [ ]:
y_premeno_test.value_counts()

OP
0     331
1      13
dtype: int64

In [ ]:
y_premeno_pred = np.where(y_premeno_pred < 0, 0, 1)
unique, counts = np.unique(y_premeno_pred, return_counts=True)
print(dict(zip(unique, counts)))

{0: 208, 1: 136}


In [ ]:
y_postmeno_test.value_counts()

OP
0     417
1     170
dtype: int64

In [ ]:
y_postmeno_pred = np.where(y_postmeno_pred < 0, 0, 1)
unique, counts = np.unique(y_postmeno_pred, return_counts=True)
print(dict(zip(unique, counts)))

{1: 587}


In [ ]:
print("폐경 전 여성, SVM")
SAMCGS(y_premeno_test.to_numpy(), y_premeno_pred)

폐경 전 여성, SVM

  Confusion Matrix:
[[200 131]
 [  8   5]]

  Accuracy: 0.596
  Precison: 0.037
  Recall  : 0.385
  F1 Score: 0.067


In [ ]:
print("폐경 후 여성, SVM")
SAMCGS(y_postmeno_test, y_postmeno_pred)

폐경 후 여성, SVM

  Confusion Matrix:
[[  0 417]
 [  0 170]]

  Accuracy: 0.290
  Precison: 0.290
  Recall  : 1.000
  F1 Score: 0.449


## XGBoost

In [ ]:
# create model instance
xgb_premeno = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgb_postmeno = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')

# fit model
xgb_premeno.fit(X_premeno_train, y_premeno_train)
xgb_postmeno.fit(X_postmeno_train, y_postmeno_train)

# make predictions
y_premeno_pred = xgb_premeno.predict(X_premeno_test)
y_postmeno_pred = xgb_postmeno.predict(X_postmeno_test)

In [ ]:
print("폐경 전 여성, XGBoost")
SAMCGS(y_premeno_test, y_premeno_pred)

폐경 전 여성, XGBoost

  Confusion Matrix:
[[330   1]
 [ 12   1]]

  Accuracy: 0.962
  Precison: 0.500
  Recall  : 0.077
  F1 Score: 0.133


In [ ]:
print("폐경 후 여성, XGBoost")
SAMCGS(y_postmeno_test, y_postmeno_pred)

폐경 후 여성, XGBoost

  Confusion Matrix:
[[389  28]
 [142  28]]

  Accuracy: 0.710
  Precison: 0.500
  Recall  : 0.165
  F1 Score: 0.248


## LGBM

In [ ]:
# # Define the LightGBM model
# params = {
#     'objective': 'binary',
#     'metric': 'auc',
#     'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

# train_data = lgb.Dataset(X_train, label=y_train)
# test_data = lgb.Dataset(X_test, label=y_test)

# # Train the model
# model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, test_data], early_stopping_rounds=10)

# # Make predictions on test set
# y_pred = model.predict(X_test)

clf_premeno = lgb.LGBMClassifier()
clf_premeno.fit(X_premeno_train, y_premeno_train) # LightGBMError: Do not support special JSON characters in feature name.
                                                  #
y_premeno_pred=clf_premeno.predict(X_premeno_test)

clf_postmeno = lgb.LGBMClassifier()
clf_postmeno.fit(X_postmeno_train, y_postmeno_train)
y_postmeno_pred=clf_postmeno.predict(X_postmeno_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

In [ ]:
print("폐경 전 여성, LGBM")
SAMCGS(y_premeno_test, y_premeno_pred)

폐경 전 여성, LGBM

  Confusion Matrix:
[[331   0]
 [ 13   0]]

  Accuracy: 0.962
  Precison: 1.000
  Recall  : 0.000
  F1 Score: 0.000


In [ ]:
print("폐경 후 여성, LGBM")
SAMCGS(y_postmeno_test, y_postmeno_pred)

폐경 후 여성, LGBM

  Confusion Matrix:
[[383  34]
 [141  29]]

  Accuracy: 0.702
  Precison: 0.460
  Recall  : 0.171
  F1 Score: 0.249


## MLP

- 이전에 가장 좋았던 코드 사용
  - 고혈압 분석 모델 참고
  - hidden:
    - layer: 5
    - dimension: 32
  - output:
    - dimmension: 1
    - activation: Sigmoid

- 내가 할 코드
  - hidden:
    - layer:
    - dimension:
  - output:
    - dimmension:
    - activation: Sigmoid


### 블로그에서 가져온 클래스
출처: https://data-science-hi.tistory.com/188


- X_premeno_train
- X_premeno_test
- y_premeno_train
- y_premeno_test

- X_postmeno_train
- X_postmeno_test
- y_postmeno_train
- y_postmeno_test

In [ ]:
# torch의 Dataset 을 상속.
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
# 학습 데이터, 시험 데이터 배치 형태로 구축하기
# premeno, postmeno 따로 생성

premeno_train_sets = TensorData(X_premeno_train_tensor, y_premeno_train_tensor)
premeno_train_loader = torch.utils.data.DataLoader(premeno_train_sets, batch_size=32, shuffle=True)

premeno_test_sets = TensorData(X_premeno_test_tensor, y_premeno_test_tensor)
premeno_test_loader = torch.utils.data.DataLoader(premeno_test_sets, batch_size=32, shuffle=False)

postmeno_train_sets = TensorData(X_postmeno_train_tensor, y_postmeno_train_tensor)
postmeno_train_loader = torch.utils.data.DataLoader(postmeno_train_sets, batch_size=32, shuffle=True)

postmeno_test_sets = TensorData(X_postmeno_test_tensor, y_postmeno_test_tensor)
postmeno_test_loader = torch.utils.data.DataLoader(postmeno_test_sets, batch_size=32, shuffle=False)

In [ ]:
# input, hidden dim 매개변수로 따로 넣어주기

class Regressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(input_dim, hidden_dim, bias=True) # 입력층(13) -> 은닉층1(50)으로 가는 연산
        self.fc2 = nn.Linear(hidden_dim, hidden_dim, bias=True) # 은닉층1(50) -> 은닉층2(30)으로 가는 연산
        self.fc3 = nn.Linear(hidden_dim, 1, bias=True) # 은닉층2(30) -> 출력층(1)으로 가는 연산
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = fn.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
        x = self.dropout(fn.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.(즉, 30개의 20%인 6개의 노드가 계산에서 제외된다.)
        x = fn.relu(self.fc3(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.

        return x

# 주의 사항
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [ ]:
mlp_premeno = Regressor(X_premeno_train.shape[1], 32)
mlp_postmeno = Regressor(X_postmeno_train.shape[1], 32)

In [ ]:
# criterion = nn.MSELoss() critation은 earlystopping을 사용할 때 쓰는 것.
objective = nn.MSELoss()
optimizer = optim.Adam(mlp_postmeno.parameters(), lr=0.001, weight_decay=1e-7)

In [ ]:
loss_pre = [] # loss를 저장할 리스트.
loss_post = []
# n = len(trainloader)
len_pre_loader = len(premeno_train_loader)
len_post_loader = len(postmeno_train_loader)

for epoch in range(400):
  running_loss = 0.0 # 한 에폭이 돌 때 그안에서 배치마다 loss가 나온다. 즉 한번 학습할 때 그렇게 쪼개지면서 loss가 다 나오니 MSE를 구하기 위해서 사용한다.

  for i, data in enumerate(postmeno_train_loader, 0): # 무작위로 섞인 32개의 데이터가 담긴 배치가 하나씩 들어온다.
    inputs, values = data # data에는 X, Y가 들어있다.
    optimizer.zero_grad() # 최적화 초기화.
    outputs = mlp_premeno(inputs) # 모델에 입력값을 넣어 예측값을 산출한다.

    loss = objective(outputs, values) # 손실함수를 계산. error 계산.
    loss.backward() # 손실 함수를 기준으로 역전파를 설정한다.

    optimizer.step() # 역전파를 진행하고 가중치를 업데이트한다.

    running_loss += loss.item() # epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.

  loss_post.append(running_loss/len_post_loader) # MSE(Mean Squared Error) 계산

In [ ]:
# model: mlp_premeno dataloader: premeno_train_loader
def evaluation(model, dataloader):

  predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서.
  actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서.

  with torch.no_grad():
    model.eval() # 평가를 할 땐 반드시 eval()을 사용해야 한다.

    for data in dataloader:
      inputs, values = data
      outputs = model(inputs)

      predictions = torch.cat((predictions, outputs), 0) # cat함수를 통해 예측값을 누적.
      actual = torch.cat((actual, values), 0) # cat함수를 통해 실제값을 누적.

  predictions = predictions.numpy() # 넘파이 배열로 변경.
  actual = actual.numpy() # 넘파이 배열로 변경.

  SAMCGS(actual.ravel().astype(np.int64), predictions.ravel().astype(np.int64))

  rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE를 계산.

  return rmse

In [ ]:
# train_rmse = evaluation(mlp_premeno, premeno_train_loader) # 원래는 이렇게 하면 안되지만, 비교를 위해서 train을 넣어서 본다.
# test_rmse = evaluation(mlp_premeno, premeno_test_loader)
test_rmse = evaluation(mlp_postmeno, postmeno_test_loader)

# print(f'train rmse:{train_rmse}')
print(f'test rmse:{test_rmse}')


  Confusion Matrix:
[[417   0]
 [170   0]]

  Accuracy: 0.710
  Precison: 1.000
  Recall  : 0.000
  F1 Score: 0.000
test rmse:0.4834246337413788
